In [240]:
import pandas as pd

In [241]:
with open("data.txt") as f:
    lines = list(map(lambda x: list(x.strip()), f.readlines()))
    df = pd.DataFrame(lines)
df

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,.,.,.,@,.,@,@,@,.,@,...,@,.,.,@,@,@,.,@,.,.
1,@,@,@,@,@,.,.,.,@,@,...,@,@,@,.,.,.,@,@,@,.
2,.,@,@,@,@,.,.,@,@,@,...,@,@,.,.,.,@,.,.,@,@
3,@,@,@,.,@,@,@,@,.,.,...,@,@,.,@,@,@,.,@,.,@
4,@,@,@,@,.,@,@,@,@,@,...,.,.,@,.,@,@,@,.,@,@
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,@,@,@,.,.,.,@,@,@,@,...,@,@,@,.,.,@,@,.,@,@
135,@,@,@,@,@,.,@,@,.,@,...,@,.,@,.,@,.,@,.,.,@
136,@,@,.,@,@,.,@,@,@,.,...,.,@,@,@,@,@,@,@,@,@
137,.,.,@,@,@,@,.,@,@,.,...,.,@,.,.,.,.,.,@,@,@


In [242]:
def get_neigbour_indizes(row, col, df):
    rows, cols = df.shape
    result = []
    for d_row in range(row-1, row + 2):
        for d_col in range(col-1, col + 2):
            if d_row == row and d_col == col:
                continue
            if d_row < 0 or d_col < 0:
                continue
            if d_row >= rows or d_col >= cols:
                continue
            result.append((d_row, d_col))
    return result

def is_wall_and_can_be_accessed(row, col, dataframe):
    character = dataframe.iloc[row, col]
    if character != "@":
        return False
        
    neighbouring_wall_counter = 0
    neighbouring_fields = get_neigbour_indizes(row, col, dataframe)
    for field in neighbouring_fields:
            if dataframe.iloc[field[0], field[1]] == '@':
                neighbouring_wall_counter += 1
    return neighbouring_wall_counter < 4

rows, cols = df.shape
accessibility_map = df.copy()

for row in range(0, rows):
    for col in range(0, cols):
        accessibility_map.iloc[row, col] = is_wall_and_can_be_accessed(row, col, df)
        
accessibility_map.to_numpy().sum()

1537

In [243]:
import itertools
import numpy as np

accessibility_map = df.copy()
board = df.copy()

inital_rolls = (board == "@").to_numpy().sum()
fields_to_check = list(itertools.product(range(0, board.shape[0]), range(0, board.shape[1])))
while len(fields_to_check) > 0:
    for field in fields_to_check:
        row, col = field
        accessibility_map.iloc[row, col] = is_wall_and_can_be_accessed(row, col, board)
    
    accessibility_map = accessibility_map == True
    coord = np.where(accessibility_map)
    updated_fields = [(int(x), int(y)) for x, y in zip(coord[0], coord[1])]
    
    potentially_impacted_fields = set(itertools.chain.from_iterable(map(lambda x: get_neigbour_indizes(x[0], x[1], board), updated_fields)))
    fields_to_check = potentially_impacted_fields
    board[accessibility_map] = "."
    
    for updated_field in updated_fields:
        row, col = updated_field
        accessibility_map.iloc[row, col] = False
    
fixed_rolls = (board == "@").to_numpy().sum()
inital_rolls-fixed_rolls

np.int64(8707)